In [1]:
# General libraries
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from matplotlib import cm
from mpl_toolkits.axes_grid1 import ImageGrid
import math

# Deep learning libraries
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import PIL



In [2]:
#Image generators for importing data and providing validation split for training
train_image = ImageDataGenerator(rescale=1/255,
                                 width_shift_range=0.05,
                                 height_shift_range=0.05,
                                 horizontal_flip=True,
                                 shear_range=10,
                                 brightness_range=[0.95,1.05],
                                 validation_split=.2)

#attempting to create validation data set without augmentations applied to the training set
#random seed is set upon generation of the data, thus should prevent data leakage
validation_image = ImageDataGenerator(rescale=1/255,
                                     validation_split=.2)

test_image = ImageDataGenerator(rescale=1/255)

In [3]:
#training data processing
train_gen = train_image.flow_from_directory(
    directory='chest_xray/train', 
    target_size=(128, 128),color_mode='grayscale',
    batch_size=32, 
    class_mode='categorical', subset='training', interpolation="lanczos",
    seed=42)

Found 4186 images belonging to 3 classes.


In [4]:
#validation data processing
val_gen = validation_image.flow_from_directory(
    directory='chest_xray/train', 
    target_size=(128, 128),color_mode='grayscale',
    batch_size=32, 
    class_mode='categorical', subset='validation', interpolation="lanczos",
    seed=42)

Found 1045 images belonging to 3 classes.


In [7]:
#testing data processing
test_gen = test_image.flow_from_directory(
    directory='chest_xray/test', 
    target_size=(128, 128), color_mode='grayscale', 
    batch_size=32, 
    class_mode='categorical', interpolation="lanczos",
    seed=42)

Found 624 images belonging to 3 classes.


In [ ]:
#initial model
model = models.Sequential()
model.add(layers.Conv2D(filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=(128, 128, 1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model.fit(train_gen, epochs=10, validation_data=val_gen, callbacks=model_checkpoint_callback)

Epoch 1/10
131/131 [==============================] - ETA: 0s - loss: 0.8505 - accuracy: 0.6149

In [ ]:
model.summary()

In [ ]:
#2nd model iteration - increased kernel size, added padding
kernel = (8, 8)

model2 = models.Sequential()
model2.add(layers.Conv2D(filters=32,
                        kernel_size=kernel,
                        activation='relu',
                        input_shape=(128, 128, 1),
                        padding='valid'))
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Conv2D(128, kernel, activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(128, kernel, activation='relu'))
model2.add(layers.Flatten())
model2.add(layers.Dense(128, activation='relu'))
model2.add(layers.Dense(3, activation='softmax'))

model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model2.fit(train_gen,epochs=10, validation_data=val_gen, callbacks=model_checkpoint_callback)

In [ ]:
model2.summary()

In [ ]:
#3rd model iteration - increased pool size
kernel = (8, 8)
pool = (3, 3)

model3 = models.Sequential()
model3.add(layers.Conv2D(filters=32,
                        kernel_size=kernel,
                        activation='relu',
                        input_shape=(128, 128, 1),
                        padding='valid'))
model3.add(layers.MaxPooling2D(pool_size=pool))

#hidden layers
model3.add(layers.Conv2D(128, kernel, activation='relu'))
model3.add(layers.MaxPooling2D(pool))
model3.add(layers.Conv2D(128, kernel, activation='relu'))
model3.add(layers.Flatten())
model3.add(layers.Dense(128, activation='relu'))
model3.add(layers.Dense(3, activation='softmax'))

model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model3.fit(train_gen,epochs=10,validation_data=val_gen)

In [ ]:
model3.summary()

In [ ]:
#4th model iteration - changed activation function for hidden layers to tanh, converted last hidden layer to dropout layer
kernel = (8, 8)
pool = (3, 3)

model4 = models.Sequential()
model4.add(layers.Conv2D(filters=32,
                        kernel_size=kernel,
                        activation='relu',
                        input_shape=(128, 128, 1),
                        padding='valid'))
#hidden layers
model4.add(layers.MaxPooling2D(pool_size=pool))
model4.add(layers.Conv2D(128, kernel, activation='tanh'))
model4.add(layers.MaxPooling2D(pool))
model4.add(layers.Conv2D(128, kernel, activation='tanh'))
model4.add(layers.Flatten())
model4.add(layers.Dense(128, activation='tanh'))
model4.add(layers.Dropout(rate=.25))
#output layer
model4.add(layers.Dense(3, activation='softmax'))

model4.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model4.fit(train_gen,epochs=10,validation_data=val_gen, callbacks=model_checkpoint_callback)

In [ ]:
model4.summary()

In [ ]:
#5th model iteration - reverted activation function changes, kept dropout layer
kernel = (8, 8)
pool = (3, 3)

model5 = models.Sequential()
model5.add(layers.Conv2D(filters=32,
                        kernel_size=kernel,
                        activation='relu',
                        input_shape=(128, 128, 1),
                        padding='valid'))
#hidden layers
model5.add(layers.MaxPooling2D(pool_size=pool))
model5.add(layers.Conv2D(128, kernel, activation='relu'))
model5.add(layers.MaxPooling2D(pool))
model5.add(layers.Conv2D(128, kernel, activation='relu'))
model5.add(layers.Flatten())
model5.add(layers.Dense(128, activation='relu'))
model5.add(layers.Dropout(rate=.25))
#output layer
model5.add(layers.Dense(3, activation='softmax'))

#compiling and fitting the model
model5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model5.fit(train_gen,epochs=10,validation_data=val_gen, callbacks=model_checkpoint_callback)

In [ ]:
model5.summary()

In [ ]:
#5th model iteration - added 1 extra convlution, reduced kernel size
kernel = (8, 8)
pool = (3, 3)

model6 = models.Sequential()
model6.add(layers.Conv2D(filters=32,
                        kernel_size=kernel,
                        activation='relu',
                        input_shape=(128, 128, 1),
                        padding='valid'))
#hidden layers
model6.add(layers.MaxPooling2D(pool_size=pool))
model6.add(layers.Conv2D(128, kernel, activation='relu'))
model6.add(layers.Conv2D(128, kernel, activation='relu'))
model6.add(layers.MaxPooling2D(pool))
model6.add(layers.Conv2D(128, kernel, activation='relu'))
model6.add(layers.Flatten())
model6.add(layers.Dense(128, activation='relu'))
model6.add(layers.Dropout(rate=.25))
#output layer
model6.add(layers.Dense(3, activation='softmax'))

#compiling and fitting the model
model6.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model6.fit(train_gen,epochs=20,validation_data=val_gen, callbacks=model_checkpoint_callback)

In [ ]:
model6.summary()

# Larger Image Sizes

Due to the granularity typically involved in human interpretation of Chest X-ray images (i.e. identifying presence and patterns of infiltrate in the lungs) we are experimenting with increasing the size of the images to achieve greater precision. This poses a greater risk of overfitting, and increases training time, however, it is possible that using convolutions and pooling, our model may be able to aggregate and interpret this finer detail if tuned appropriately.

In [ ]:
#training data processing using half length and width of original image size
train_gen_0 = train_image.flow_from_directory(
    directory='chest_xray/train', 
    target_size=(534, 381),color_mode='grayscale',
    batch_size=64, 
    class_mode='categorical', subset='training', interpolation="lanczos",
    seed=42)

#validation data processing using half length and width of original image size
val_gen_0 = validation_image.flow_from_directory(
    directory='chest_xray/train', 
    target_size=(534, 381),color_mode='grayscale',
    batch_size=128, 
    class_mode='categorical', subset='validation', interpolation="lanczos",
    seed=42)

In [ ]:
#initial model using half length and width of original image size
model7 = models.Sequential()

#input layer
model7.add(layers.Conv2D(filters=3,
                        kernel_size=(20, 20),
                        activation='swish',
                        input_shape=(534, 381, 1)))
model7.add(layers.Flatten())

#hidden layers
model7.add(layers.Dense(128, activation='swish'))
model7.add(layers.Dense(64, activation='swish'))
model7.add(layers.Dense(32, activation='swish'))

#output layer
model7.add(layers.Dense(3, activation='softmax'))

#compile and compute
model7.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'AUC'])
#creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model7.fit(train_gen_0, epochs=20, validation_data=val_gen_0, callbacks=model_checkpoint_callback)

# Deep Models

In [ ]:
#Going Deeper
model8 = models.Sequential()

#input layer
model8.add(layers.Conv2D(filters=32,
                        kernel_size=(20, 20),
                        activation='relu',
                        input_shape=(128, 128, 1)))
model8.add(layers.MaxPooling2D(pool_size=(2, 2)))

#hidden layers
model8.add(layers.Conv2D(32, (2, 2), activation='relu'))
model8.add(layers.MaxPooling2D((2, 2)))
model8.add(layers.Conv2D(32, (2, 2), activation='relu'))
model8.add(layers.Flatten())
model8.add(layers.Dense(64, activation='relu'))
model8.add(layers.Dense(32, activation='relu'))
model8.add(layers.Dense(16, activation='relu'))
model8.add(layers.Dense(8, activation='relu'))

#output layer
model8.add(layers.Dense(3, activation='softmax'))

#compile and compute
model8.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'AUC'])
#creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model8.fit(train_gen, epochs=50, validation_data=val_gen, callbacks=model_checkpoint_callback)

In [ ]:
model8.summary()

In [ ]:
#Going Even Deeper
model9 = models.Sequential()

#input layer
model9.add(layers.Conv2D(filters=32,
                        kernel_size=(20, 20),
                        activation='relu',
                        input_shape=(128, 128, 1)))
model9.add(layers.MaxPooling2D(pool_size=(2, 2)))

#hidden layers
model9.add(layers.Conv2D(32, (2, 2), activation='relu'))
model9.add(layers.Flatten())
#dense layers for interpretation
model9.add(layers.Dense(64, activation=layers.LeakyReLU()))
model9.add(layers.Dense(64, activation='relu'))
model9.add(layers.Dense(64, activation=layers.LeakyReLU()))
model9.add(layers.Dense(64, activation='relu'))
model9.add(layers.Dropout(.5))
model9.add(layers.Dense(64, activation=layers.LeakyReLU()))
model9.add(layers.Dense(64, activation='relu'))
model9.add(layers.Dropout(.5))
model9.add(layers.Dense(64, activation=layers.LeakyReLU()))
model9.add(layers.Dense(64, activation='relu'))

#output layer
model9.add(layers.Dense(3, activation='softmax'))

#compile and compute
model9.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'AUC'])
#creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model9.fit(train_gen, epochs=50, validation_data=val_gen, callbacks=model_checkpoint_callback)

In [ ]:
#Went too deep - trying new activation function in hidden layers
model0 = models.Sequential()

#input layer
model0.add(layers.Conv2D(filters=16,
                        kernel_size=(20, 20),
                        activation='relu',
                        input_shape=(128, 128, 1)))
model0.add(layers.MaxPooling2D(pool_size=(2, 2)))

#hidden layers
model0.add(layers.Conv2D(32, (2, 2), activation='swish'))
model0.add(layers.MaxPooling2D((2, 2)))
model0.add(layers.Conv2D(32, (2, 2), activation='swish'))
model0.add(layers.Flatten())
model0.add(layers.Dense(64, activation='swish'))
model0.add(layers.Dense(32, activation='swish'))
model0.add(layers.Dense(16, activation='swish'))
model0.add(layers.Dense(8, activation='swish'))

#output layer
model0.add(layers.Dense(3, activation='softmax'))

#compile and compute
model0.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'AUC'])
#creating a save file for this model
model_checkpoint_callback = ModelCheckpoint(filepath="./Checkpoints",
                                           save_weights_only=True,
                                           monitor='val_accuracy',
                                           mode='max',
                                           save_best_only=True)

model0.fit(train_gen, epochs=50, validation_data=val_gen, callbacks=model_checkpoint_callback)